# Casual Tree

Random Forestをカーネル重み付けとみなす視点に、Robinson (1988)の部分線形モデルを合わせたもの

[一般化ランダムフォレストの理論と統計的因果推論への応用 - Speaker Deck](https://speakerdeck.com/tomoshige_n/ban-hua-randamuhuoresutonoli-lun-totong-ji-de-yin-guo-tui-lun-henoying-yong?slide=42)


## 応用研究



### 小売業においてクーポン配布のCATEを推定

[How causal machine learning can leverage marketing strategies: Assessing and improving the performance of a coupon campaign | PLOS One](https://journals.plos.org/plosone/article?id=10.1371/journal.pone.0278937)

- Kaggle公開データを使ってCATEを推定
- 配布した5種類のクーポンのうち、ドラッグストア商品券と食品カテゴリ券の2種だけが有意に売上増効果を持つことが分かった。
- 他のクーポンは統計的に効果が確認できず、無効な施策だった。
- また効果の異質性分析から、「ドラッグストア券は事前購買額の大きい優良顧客に特に効果的」「一般食品券は購買額の少ないライト層に効果的」といったセグメント別の効果差が判明した
- 例えば、既存ヘビーユーザーには日用品系クーポンで追加購買を促進でき、ライトユーザーには普段買わない食品クーポンで来店を促すと効果的。
- この結果を踏まえ、クーポン配布の対象を効果の高いセグメントに絞る最適施策を導出したところ、全顧客に配布する場合と比べ費用対効果が大幅に向上した（少数配布で売上効果の大半を実現）。因果フォレストを用いることで、マーケティングROIを最大化する配布戦略の策定が可能になることを示した。


### Amazonの顧客ターゲティング

[[2409.13847] Segment Discovery: Enhancing E-commerce Targeting](https://arxiv.org/abs/2409.13847)

方法
- Amazon社内の大規模実験データ（オンラインサービス等のABテスト複数）。例：ある定額サービスで解約防止のプロモメッセージを送る実験（数十万人規模）。
- まずランダム化比較試験から顧客毎のCATEをCausal Forestやuplift modelで推定。次に企業側の制約（予算・接触可能数）を組み込んだ最適化アルゴリズム（Large-Scale Budget-Constrained Causal Forest）で最大効果のターゲット集合を選定。

結果
- 従来は、例えば離脱率の高い順に顧客に介入するなど予測モデルに基づくターゲティングが行われていた。しかし実験データを分析したところ、離脱リスクが極端に高い顧客層では、プロモメッセージを送っても効果が無いばかりか逆効果（介入で離脱率が悪化）になる場合があることが判明
- 一方、リスクが中程度の層に大きなプラス効果が見られるなど、どの層に介入すべきかは単純な予測スコア順とは一致しないことが示された。
- Causal Forestを用いるターゲティングでは、こうした「介入効果の高い層」を的確に抽出できる。実際、**あるサービスの例では従来ルールでは離脱スコア下位39.1%を一律介入対象としていたが、因果モデルに基づき効果の見込める顧客だけを選別したところ、むしろ全体の半数以下の配信で離脱抑止効果の75%以上を達成**できた。
- このアプローチにより、顧客体験を損ねる無駄な介入を避けつつ、限られたマーケ予算で最大限の効果を上げることが可能となる。Amazon社の研究では複数のキャンペーンで有効性が検証されており、大規模パーソナライズドマーケティングへの因果機械学習の実用性を示すケースとなっている。

### 「少しずつ頻繁に」行う宿題の学習効果 McJames et al. (2024)

- [Little and often: Causal inference machine learning demonstrates the benefits of homework for improving achievement in mathematics and science - ScienceDirect](https://www.sciencedirect.com/science/article/pii/S0959475224000951?via%3Dihub)
- [Study reveals impact of homework on student achievement in math and science](https://phys.org/news/2024-09-reveals-impact-homework-student-math.html)

方法：

- 国際学力調査TIMSS 2019のデータから、アイルランドの中学2年生4,118人の数学・理科成績と家庭学習時間を分析
- Bayesian Causal Forests で、宿題の頻度・長さを疑似介入変数として成績への因果効果を推定

結果：
- 週あたりの宿題実施回数を増やすことが学力向上に効果的である一方、1回あたりの宿題時間を延ばしても効果は頭打ちになる傾向が示された
- 15分程度の短い課題でも長時間の課題と同等の効果があり、頻度を上げて宿題を小分けにする「Little and Often」戦略が有効であることがデータから裏付けられた。
- **数学では「毎日少し」の宿題が最も成績を伸ばし** 、理科では週3～4回程度が効果的。
- また従来の通説では家庭環境により宿題効果に差が出ると考えられていたが、本解析ではどの社会経済層の生徒も宿題の恩恵を等しく享受しており、一部の層だけが得をするわけではないことが示された。
